In [1]:
import os
os.chdir('/Users/amorris/Documents/sms-pumping')

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/Users/amorris/Documents/sms-pumping/.env")
import os

import pandas as pd
import numpy as np
import prestodb
import os
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from scipy.stats import beta
import pickle
import looker_sdk

from notebook_auto_alert import done
pd.options.display.max_columns=200
pd.options.display.max_rows = 200


presto_host = os.getenv('presto_host')
presto_port = os.getenv('presto_port')
username = os.getenv('presto_username')
password = os.getenv('presto_password')

conn = prestodb.dbapi.connect(
            host=presto_host,
            port=presto_port,
            user=username,
            catalog='hive',
            schema='public',
            http_scheme='https',
            auth=prestodb.auth.BasicAuthentication(username, password),
        )

%matplotlib inline



✓ Auto-error detection enabled - will alert if any cell fails!


In [3]:
def get_presto_data(query, presto_connection):
    cur = presto_connection.cursor()
    t1 = pd.to_datetime('now', utc=True)
    cur.execute(query)
    data = cur.fetchall()
    cols = [desc[0] for desc in cur.description]
    df = pd.DataFrame(data, columns=cols)
    t2 = pd.to_datetime('now', utc=True)
    print('Query took {} seconds to complete'.format((t2-t1).total_seconds()))
    return df

# Set Parameters

In [ ]:
# Parameters

looker = ""

In [ ]:
import re

split_marker = "-- sql for creating the total and/or determining pivot columns"
if split_marker in looker:
    looker = looker.split(split_marker)[0]

if 'AS iso_country_code' not in looker:
    select_match = re.search(r'SELECT\s+(.*?)(?=\s+FROM)', looker, flags=re.DOTALL)
    if select_match:
        select_block = select_match.group(1).strip()
        updated_select_block = select_block + ',\n    billable_items.iso_country_code AS iso_country_code'
        looker = looker.replace(select_block, updated_select_block)

group_by_match = re.search(r'GROUP BY\s+(.*?)(?=\s+ORDER BY|\s+LIMIT|$)', looker, flags=re.DOTALL)
if group_by_match:
    group_by_block = group_by_match.group(1).strip()
    if 'billable_items.iso_country_code' not in group_by_block:
        updated_group_by = group_by_block + ',\n    billable_items.iso_country_code'
        looker = looker.replace(group_by_block, updated_group_by)

looker_df = get_presto_data(looker, presto_connection=conn)

country_list = looker_df['iso_country_code'].unique().tolist()
country_string = ', '.join(f"'{country}'" for country in country_list)

account_sid_pattern = r"account_sid\s*\)\s*=\s*'([^']+)'"
account_sid_match = re.search(account_sid_pattern, looker)
if account_sid_match:
    account_sid = account_sid_match.group(1)
    print("Account SID found:", account_sid)
else:
    account_sid = None
    print("No match found for account SID")

date_range_pattern = r"TIMESTAMP\s*'([^']+)'"
date_range_match = re.findall(date_range_pattern, looker)

if len(date_range_match) >= 2:
    start_date = date_range_match[0]
    end_date = date_range_match[1]
    print("Start Date:", start_date)
    print("End Date:", end_date)
else:
    print("Could not extract date range.")

Query took 246.620084 seconds to complete
Account SID found: ACXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Start Date: 2025-07-01
End Date: 2025-09-30


In [5]:
mcc = f'''


select

mcc

from "public".mcc_country

where two_letter_country_code in ({country_string})


'''

mcc_df = get_presto_data(mcc, presto_connection=conn)


mcc_list = mcc_df['mcc'].tolist()


#mcc_list

Query took 4.059724 seconds to complete


In [6]:
#account_sid = 'ACXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [7]:
end_date = pd.to_datetime(end_date)
end_date = end_date - pd.Timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')
print(start_date)
print(end_date)
print(account_sid)
mcc_string = ', '.join(f"'{mcc}'" for mcc in mcc_list)
print(mcc_string)

2025-07-01
2025-09-29
ACXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
'316', '744', '628', '520', '206', '640', '746', '610', '428', '530', '213', '311', '278', '419', '708', '246', '553', '466', '374', '622', '350', '310', '748', '541', '425', '352', '342', '714', '220', '734', '630', '550', '208', '280', '544', '340', '276', '360', '537', '611', '230', '240', '440', '282', '259', '619', '358', '441', '616', '639', '244', '710', '226', '222', '643', '604', '310', '505', '416', '257', '552', '405', '730', '659', '602', '617', '414', '372', '712', '546', '364', '286', '218', '363', '274', '250', '348', '732', '312', '654', '621', '609', '248', '738', '706', '242', '637', '401', '344', '547', '627', '330', '231', '633', '450', '655', '510', '455', '204', '302', '418', '624', '415', '202', '338', '313', '615', '370', '434', '214', '623', '311', '648', '502', '266', '238', '366', '310', '216', '404', '551', '424', '724', '515', '736', '528', '605', '427', '262', '232', '539', '647', '429', '645', '293'

## FG Settings

In [8]:
fg_settings = f'''


SELECT

fraud_guard_enabled
,mcc
,reason
,protection_mode
,count(*) as count


FROM
hive.accsecai.dsg_fds_raw_events a
left JOIN hive.public.rawpii_accounts par on par.sid = a.account_sid

WHERE
data_load_date >= date '{start_date}'
and data_load_date <= date '{end_date}'
and (account_sid = '{account_sid}'
or par.parent_account_sid = '{account_sid}')
and mcc in ({mcc_string})
and mcc not in ('310', '311', '312', '313', '302')
and product_id = 'Verify'
and reason <> 'duplicate_request_skipped'
and channel = 'sms'
group by 1,2,3,4
order by 5 desc


'''

fg_settings_df = get_presto_data(fg_settings, presto_connection=conn)

fg_settings_df

KeyboardInterrupt: 

In [ ]:
## Was FG enabed
pct_enabled = fg_settings_df[fg_settings_df['fraud_guard_enabled'] == True]['count'].sum()/fg_settings_df['count'].sum()

## additional metrics
enabled_df = fg_settings_df[fg_settings_df['fraud_guard_enabled'] == True]
scanned_df = enabled_df[(enabled_df['reason'] != 'pass_country_level_config_request') & (enabled_df['reason'] != 'risk_check_disabled')]

## mode splits
basic_pct_enabled = scanned_df[scanned_df['protection_mode'] == 'basic']['count'].sum()/scanned_df['count'].sum()
standard_pct_enabled = scanned_df[scanned_df['protection_mode'] == 'standard']['count'].sum()/scanned_df['count'].sum()
max_pct_enabled = scanned_df[scanned_df['protection_mode'] == 'max']['count'].sum()/scanned_df['count'].sum()

##country disabled
country_disabled = enabled_df[enabled_df['reason'] == 'pass_country_level_config_request']['count'].sum()/fg_settings_df['count'].sum()

## risk check disabled
risk_check_disabled = enabled_df[enabled_df['reason'] == 'risk_check_disabled']['count'].sum()/fg_settings_df['count'].sum()

##pct scanned
percent_scanned = enabled_df[(enabled_df['reason'] != 'pass_country_level_config_request') & (enabled_df['reason'] != 'risk_check_disabled')]['count'].sum()/fg_settings_df['count'].sum()



## Account Spend

In [ ]:
parent = f'''

select

parent_account_sid

from hive.public.rawpii_accounts

where sid = '{account_sid}'

'''

parent_df = get_presto_data(parent, presto_connection=conn)

parent_account_sid = parent_df['parent_account_sid'].iloc[0]




Query took 3.930222 seconds to complete


In [ ]:
mrr_account_sid = account_sid if parent_account_sid in [None, "None"] else parent_account_sid

mrr_account_sid

'ACXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [ ]:
mrr = f'''

select
    '{account_sid}' as account_sid,
    sum(total)/3 as mrr
from public.counters_by_billable_item_complete
    where account_sid = '{mrr_account_sid}'
    and "date" >= DATE_ADD('day', -90, DATE '{start_date}')
    and "date" < DATE '{start_date}'
    and subtype in ('sale')
group by 1
order by 1,2

'''

mrr_df = get_presto_data(mrr, presto_connection=conn)

mrr_df['mrr'] = mrr_df['mrr'].astype(float).astype(int)

Query took 12.271412 seconds to complete


In [ ]:
true_cost = f'''

-- Provider Quantities and Cost for Verify (Optimized)
WITH

-- 1. Filter comms table early
filtered_comms AS (
    SELECT
        id,
        "to" as phone_number,
        mcc,
        mnc,
        created_at_utc,
        account_sid,
        scope_id,
        data_load_date,
        error_code,
        -- for ease, keep these for joining
        lower(channel) as channel
    FROM
        hive.public.accsec_verify_comms a
        left JOIN hive.public.rawpii_accounts par on par.sid = a.account_sid
    WHERE
        data_load_date >= DATE '{start_date}'
        AND data_load_date <= DATE '{end_date}'
        and (account_sid = '{account_sid}'
        or par.parent_account_sid = '{account_sid}')
        AND mcc in ({mcc_string})
        AND lower(channel) = 'sms'
),

-- 2. Filter rawpii_sms_kafka table early (by SID and date)
filtered_sms AS (
    SELECT
        sid,
        provider_sid,
        price,
        error_code,
        cast(price as double) * -1.0 as price_transformed,
        data_load_date
    FROM
        hive.public.rawpii_sms_kafka
    WHERE
        data_load_date >= DATE '{start_date}'
        and data_load_date <= DATE '{end_date}'
        and mcc in ({mcc_string})

),

-- 3. Join the (much smaller) tables before further joins
the_data AS (
    SELECT
        sms.provider_sid,
        p.friendly_name AS provider_name,
        p.provider_organization_sid,
        po.friendly_name AS provider_organization_name,
        sms.error_code,
        comms.phone_number,
        substr(replace(comms.phone_number, '+'), 1, length(replace(comms.phone_number, '+'))-3) AS onek_prefix,
        substr(replace(comms.phone_number, '+'), 1, length(replace(comms.phone_number, '+'))-4) AS tenk_prefix,
        comms.mcc,
        comms.mnc,
        comms.created_at_utc,
        comms.account_sid,
        comms.scope_id,
        comms.data_load_date,
        CASE WHEN sms.price IS NULL THEN 1 ELSE 0 END AS price_is_null,
        sms.price_transformed AS price
    FROM
        filtered_comms comms
        LEFT JOIN filtered_sms sms ON comms.id = sms.sid
        LEFT JOIN hive.public.providers p ON p.sid = sms.provider_sid
        LEFT JOIN hive.public.provider_organizations po ON p.provider_organization_sid = po.sid
),

provider_mcc_data AS (
    SELECT
        provider_organization_name,
        provider_sid,
        mcc,
        mnc,
        COUNT(phone_number) AS record_count,
        COUNT(DISTINCT phone_number) AS distinct_pn,
        SUM(CASE WHEN error_code != 30450 THEN price_is_null ELSE 0 END) AS null_price_count,
        SUM(price) AS customer_cost
    FROM
        the_data
    GROUP BY
        provider_organization_name,
        provider_sid,
        mcc,
        mnc
),

provider_cogs AS (
    SELECT
        po.friendly_name,
        psms.provider_sid,
        psms.mcc,
        psms.mnc,
        effective_rate,
        billing_type,
        currency,
        psms.created_utc,
        ROW_NUMBER() OVER (PARTITION BY psms.provider_sid, mcc, mnc ORDER BY psms.created_utc DESC) AS rank
    FROM
        public.messaging_reporting_rates psms
        LEFT JOIN hive.public.providers p ON p.sid = psms.provider_sid
        LEFT JOIN hive.public.provider_organizations po ON p.provider_organization_sid = po.sid
    WHERE
        effective_rate IS NOT NULL
),

latest_provider_cogs AS (
    SELECT *
    FROM provider_cogs
    WHERE rank = 1
),

penultimate AS (
    SELECT
        sms.provider_organization_name,
        sms.provider_sid,
        sms.mcc,
        sms.mnc,
        p.effective_rate AS provider_rate,
        record_count,
        distinct_pn,
        null_price_count,
        customer_cost AS customer_cost_w_missing_px,
        CAST(customer_cost AS double) / NULLIF(CAST(record_count AS double) - CAST(null_price_count AS double), 0) AS avg_cost_provider_mcc,
        (CAST(customer_cost AS double) / NULLIF(CAST(record_count AS double) - CAST(null_price_count AS double), 0)) * CAST(null_price_count AS double) AS inferred_missing_px_cost,
        ((CAST(customer_cost AS double) / NULLIF(CAST(record_count AS double) - CAST(null_price_count AS double), 0)) * CAST(null_price_count AS double)) + CAST(customer_cost AS double) AS total_customer_cost,
        CAST(p.effective_rate AS double) * CAST(record_count AS double) AS provider_cogs
    FROM
        provider_mcc_data sms
        LEFT JOIN latest_provider_cogs p
            ON sms.provider_sid = p.provider_sid
            AND sms.mcc = p.mcc
            AND sms.mnc = p.mnc
)

SELECT
    provider_organization_name,
    SUM(record_count) AS record_count,
    SUM(distinct_pn) AS distinct_pn,
    SUM(customer_cost_w_missing_px) AS customer_cost_w_missing_px,
    SUM(inferred_missing_px_cost) AS inferred_missing_px_cost,
    SUM(total_customer_cost) AS total_revenue,
    SUM(provider_cogs) AS total_twilio_cogs,
    SUM(total_customer_cost) - SUM(provider_cogs) AS total_twilio_profit
FROM
    penultimate
WHERE
    customer_cost_w_missing_px > 0
GROUP BY
    1



'''

true_cost_df = get_presto_data(true_cost, presto_connection=conn)

true_cost_df.sort_values(by = 'record_count', ascending = False).head(10)

PrestoQueryError: PrestoQueryError(type=Query exhausted resources at this scale factor, name=1, message="Query exhausted resources at this scale factor", query_id=f87ce215-53d0-42ca-b656-b4370b952994)

In [ ]:
asid_df = mrr_df.copy()
asid_df['fraud_start'] = start_date
asid_df['fraud_end'] = end_date
asid_df['fraud_total'] = looker_df['counters_by_billable_item.total_sum'].astype(float).sum()

asid_df['rev'] = int(true_cost_df['total_revenue'].sum() or 0)
asid_df['cogs'] = int(true_cost_df['total_twilio_cogs'].sum() or 0)
asid_df['GP'] = (asid_df['rev'] - asid_df['cogs']).astype(float).astype(int)
asid_df['GM'] = (asid_df['GP']/asid_df['rev']).astype(float).round(3)
asid_df['Algo GP'] = int(true_cost_df[true_cost_df['provider_organization_name'] == 'SOATSO AB']['total_revenue'].sum() or 0)


asid_df

,account_sid,mrr,fraud_start,fraud_end,fraud_total,rev,cogs,GP,GM,Algo GP
0,ACXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX,3,2025-12-01,2025-12-18,9218.1782,9117,3023,6094,0.668,2772


## Final Summary

In [ ]:
## blocking info

blocking = f'''


SELECT

protection_mode,
count(*) as count,
ROUND(CAST(SUM(CASE WHEN block = true THEN 1 ELSE 0 END) AS DOUBLE) / COUNT(*), 3) AS block_rate

FROM
hive.accsecai.dsg_fds_raw_events a

left JOIN hive.public.rawpii_accounts par on par.sid = a.account_sid

WHERE
data_load_date >= date '{start_date}'
and data_load_date <= date '{end_date}'
and (account_sid = '{account_sid}'
or par.parent_account_sid = '{account_sid}')
and mcc in ({mcc_string})
and product_id = 'Verify'
and channel = 'sms'
and fraud_guard_enabled = true
and reason <> 'risk_check_disabled'
and reason <> 'duplicate_request_skipped'
and reason <> 'pass_country_level_config_request'
group by 1
order by 2 desc


'''

blocking_df = get_presto_data(blocking, presto_connection=conn)


ConnectTimeout: HTTPSConnectionPool(host='odin.prod.twilio.com', port=8443): Max retries exceeded with url: /v1/statement/queued/f9be958e-e528-4c81-8134-1df19809614b/1?slug=64334f25dd36369c6acd14f02143e2ca&applicationNamePrefix=athena&isSelectQuery=true&shouldSkipFirstRow=true&workgroup=adhoc_primary_wg_adhoc&retryCount=7 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x15f454e60>, 'Connection to odin.prod.twilio.com timed out. (connect timeout=30.0)'))

## Summary Info

In [ ]:
total_msg = blocking_df['count'].sum()

scanned_fraud_scalar = (asid_df['fraud_total'] * percent_scanned).sum()

scanned_fraud_scalar

scanned_msg_scalar = total_msg * percent_scanned

In [ ]:
import pandas as pd

mode_names = ['basic', 'standard', 'max']
pct_enableds = [basic_pct_enabled, standard_pct_enabled, max_pct_enabled]

#### expected block rates by mode
exp_block_rates = [0.80, 0.90, 1]

modes_df = pd.DataFrame({
    'mode': mode_names,
    'traffic_prop': pct_enableds
})

modes_df['fraud_cost'] = modes_df['traffic_prop'] * scanned_fraud_scalar

modes_df['msg_count'] = (modes_df['traffic_prop'] * scanned_msg_scalar).fillna(0).astype(int)

output_df = modes_df.merge(
    blocking_df.rename(columns={'protection_mode': 'mode'}),
    on='mode', how='left'
)

output_df['traffic_prop'] = output_df['traffic_prop'].astype(float)
output_df['exp_block_rates'] = exp_block_rates
output_df['unit_cost'] = output_df['fraud_cost']/(output_df['msg_count'] - (output_df['msg_count'] *(output_df['block_rate'])))    
output_df['impl_total_cost'] = output_df['unit_cost'] * output_df['msg_count']
output_df['credit'] = (output_df['impl_total_cost'] * output_df['exp_block_rates']) - (output_df['impl_total_cost'] - output_df['fraud_cost'])
output_df['traffic_prop'] = output_df['traffic_prop'].map(lambda x: f"{x:.3f}")
output_df['fraud_cost'] = output_df['fraud_cost'].map(lambda x: f"${x:,.2f}")
output_df['block_rate'] = output_df['block_rate'].map(lambda x: f"{x:.3f}")
output_df['credit'] = output_df['credit'].map(lambda x: f"${x:,.2f}")

keep = ['mode', 'traffic_prop', 'msg_count', 'fraud_cost', 'block_rate', 'credit']
output_df = output_df[keep]

In [ ]:
## config summary

print("="*35)
print(" FG Settings Summary")
print("="*35)
print(f'Percent Scanned:   {percent_scanned:.2%}')
print(f'FG enabled:        {pct_enabled:.2%}')
print(f'Country disabled:  {country_disabled:.2%}')
print(f'Risk check disabled: {risk_check_disabled:.2%}')

print("\n" + "-"*35)
print(" Breakdown by Mode")
print("-"*35)
print(f'Basic mode:        {basic_pct_enabled:.2%}')
print(f'Standard mode:     {standard_pct_enabled:.2%}')
print(f'Max mode:          {max_pct_enabled:.2%}')

 FG Settings Summary
Percent Scanned:   99.98%
FG enabled:        100.00%
Country disabled:  0.02%
Risk check disabled: 0.00%

-----------------------------------
 Breakdown by Mode
-----------------------------------
Basic mode:        99.98%
Standard mode:     0.02%
Max mode:          0.00%


In [ ]:
## malfunction amounts if applicable

output_df

,mode,traffic_prop,msg_count,fraud_cost,block_rate,credit
0,basic,1.000,35644,"$7,526.30",0.416,"$4,948.80"
1,standard,0.000,7,$1.69,1.000,$nan
2,max,0.000,0,$0.00,0.000,$nan


In [ ]:
## cost breakdown 

asid_df

,account_sid,mrr,fraud_start,fraud_end,fraud_total,rev,cogs,GP,GM,Algo GP
0,ACXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX,81,2025-11-04,2025-11-08,7529.6739,7564,3940,3624,0.479,60


In [ ]:
credit_amount = output_df['credit'].sum()

In [ ]:
#import pandas as pd
#from sklearn.ensemble import RandomForestRegressor
#import joblib


#refund_df = pd.read_csv('Refund Data.csv')

#feature_cols = ['mrr', 'fraud_total', 'rev', 'cogs', 'GP', 'GM', 'Algo GP', 'incr_ARR', 'enabled_pre_fraud']
#numeric_cols = ['mrr', 'fraud_total', 'rev', 'cogs', 'GP', 'GM', 'Algo GP', 'incr_ARR', 'credit']  # credit is target


#for col in numeric_cols:
#    refund_df[col] = refund_df[col].replace({'[,$]': ''}, regex=True).astype(float)


#refund_df['enabled_pre_fraud'] = refund_df['enabled_pre_fraud'].map({'Yes': 1, 'No': 0})


#X = refund_df[feature_cols]
#y = refund_df['credit']


#model = RandomForestRegressor(random_state=42)
#model.fit(X, y)


#joblib.dump(model, 'rf_model.joblib')
#print("Model saved as rf_model.joblib")

In [ ]:

done()

In [ ]:
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error

#refund_df = pd.read_csv('Refund Data.csv')

#feature_cols = ['mrr', 'fraud_total', 'rev', 'cogs', 'GP', 'GM', 'Algo GP', 'incr_ARR', 'enabled_pre_fraud']

#refund_df['enabled_pre_fraud'] = refund_df['enabled_pre_fraud'].map({'Yes': 1, 'No': 0})
#
#X = refund_df[feature_cols]
#y = refund_df['credit']



In [ ]:
#refund_df